Questions
1. how to know the available tables in the database?
2. how to connect to the console, tv screen? 

### Import packages 

In [1]:
import sqlite3
import pandas as pd

### Connect to database - there are two relevant tables ('author' and 'books')

In [18]:
conn = sqlite3.connect('amazon_db.db')
c = conn.cursor()

``` python
# try fetch tables in the database

c.execute('SELECT name, type FROM amazon_db') 
print(c.fetchall())

> OperationalError: no such table: amazon_db
#amazon_db is database, not schema, need to know the schema as (look-up table)         
```

In [23]:
c.execute("""SELECT * FROM books 
LIMIT 5; """) #exercute query to retrieve desired data

df_test = pd.DataFrame(c.fetchall()) # 
df_test

,0,1
0,A Elephant Ate Under A Fish,4226324
1,A Boy Jumped Outside A Lake,4048899
2,One Man Hopped Under The Tree,774329
3,A Person Threw Behind The Notebook,1359606
4,The Tree Ran Through A Mountain,4703380


In [24]:
c.description

(('books', None, None, None, None, None, None),
 ('num_copies', None, None, None, None, None, None))

In [22]:
c.execute("""SELECT * FROM books
LIMIT 5; """)  # need to re-execute this line, otherwise error

df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description] #rename col names use c.description 
df

,books,num_copies
0,A Elephant Ate Under A Fish,4226324
1,A Boy Jumped Outside A Lake,4048899
2,One Man Hopped Under The Tree,774329
3,A Person Threw Behind The Notebook,1359606
4,The Tree Ran Through A Mountain,4703380


In [5]:
c.execute("""SELECT * FROM author
LIMIT 5; """);

In [6]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,book
0,David Taylor,A Guy Dove Outside The Desk
1,Sidney Osborne,A Fish Hopped Through A Lake
2,Robert Martin,The Woman Winced Outside A Desk
3,Greg Simpson,A Banana Winced Outside The Banana
4,Edgar Williamson,A Person Ate Behind The Potato


### Use SQL to answer the following questions: 

1. How many rows are in the 'author' table? 

2. How many authors have the first name 'David'?  

3. How many books did each author write? 

4. Use both the author table and books table to answer: how many copies were sold by authors named David 


    a. how many copies for each of the Davids
    
    
    b. how many copies in total for all Davids

In [29]:
# 1. How many rows are in the 'author' table?
c.execute("""SELECT count(*) 
            FROM author;""")
c.fetchall()

[(100000,)]

```
col name & table can be the same
```

In [31]:
# 2. How many authors have the first name 'David'?
c.execute(
"""SELECT count(author)
FROM author
WHERE author LIKE 'David%'
""")
c.fetchall()

[(1335,)]

In [34]:
# 3. How many books did each author write?
c.execute(
"""
SELECT author, COUNT(book) as book_count
FROM author
GROUP BY author
ORDER By book_count DESC
""")
df_book_count = pd.DataFrame(c.fetchall())
df_book_count.columns = ['author','book_count']
df_book_count.head()

,author,book_count
0,Robert Williams,25
1,James Smith,24
2,William Smith,22
3,Robert Johnson,21
4,Mary Williams,19


In [35]:
# 4. Use both the author table and books table to answer: how many copies were sold by authors named David
''' Logic 
1. primary table: author  (col: author, book)
2. left join table: books (col: books, num_copies)
3. join on key: author.book = books.books 
4.a
    4. groupby author name 
    5. HAVING condition: author first name  = 'David'
4.b
    4. WHERE condition: author first name  = 'David'
'''

In [40]:
# 4.a: how many copies for each of the Davids
c.execute(
    """
    SELECT author, SUM(books.num_copies) as book_copies
    FROM author
    LEFT JOIN books
    ON author.book = books.books
    GROUP BY author.author
    HAVING author.author LIKE 'David %'
    ORDER BY book_copies DESC
    """)
df_David_any = pd.DataFrame(c.fetchall())
df_David_any.columns = ['author','book_copies']

In [41]:
df_David_any.head()

,author,book_copies
0,David Smith,31792538.0
1,David Jones,20043716.0
2,David Brown,16819141.0
3,David Walter,16617115.0
4,David Taylor,16173228.0


In [42]:
df_David_any.tail()

,author,book_copies
1018,David Wiford,NaN
1019,David Wilhite,NaN
1020,David Willian,NaN
1021,David Yamamoto,NaN
1022,David Zink,NaN


In [45]:
# 4.b: how many copies in total for all Davids

c.execute(
    """
    SELECT SUM(books.num_copies)
    FROM author
    LEFT JOIN books
    ON author.book = books.books
    WHERE author.author LIKE 'David %'
    """)
print(f'total book copies for al Davids: {c.fetchall()[0][0]}')

total book copies for al Davids: 2610436149
